In [133]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [134]:
original_df = pd.read_csv("GlobalLandTemperaturesByCountry+GHG.csv").drop(columns=["Unnamed: 0"])
original_df.head()

,year,Country,continent,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff,Code,gdp
0,1900,AFGHANISTAN,Asia,13.749333,0.748833,33.00,65.0,0.0,0.0,NaN,NaN
1,1900,ALBANIA,Europe,13.068583,0.723417,41.00,20.0,0.0,0.0,NaN,NaN
2,1900,ALGERIA,Africa,22.864167,0.826667,28.00,3.0,0.0,0.0,NaN,NaN
3,1900,AMERICAN SAMOA,Oceania,26.273500,0.891000,-14.33,-170.0,NaN,0.0,NaN,NaN
4,1900,ANDORRA,Europe,11.348333,0.542167,42.50,1.5,0.0,0.0,NaN,NaN


In [135]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24342 entries, 0 to 24341
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           24342 non-null  int64  
 1   Country                        24342 non-null  object 
 2   continent                      24342 non-null  object 
 3   AverageTemperature             24341 non-null  float64
 4   AverageTemperatureUncertainty  24341 non-null  float64
 5   Latitude                       24342 non-null  float64
 6   Longitude                      24342 non-null  float64
 7   GHG                            22458 non-null  float64
 8   temp_diff                      24341 non-null  float64
 9   Code                           9396 non-null   object 
 10  gdp                            7323 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 2.0+ MB


In [136]:
original_df.dropna(inplace = True)

In [137]:
#def get_temperature_diff_between(df_by_year,start_date,end_date):
   # df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
     #                   (df_by_year["year"] <= end_date)].groupby(["Country","continent"]).mean().reset_index()
    
  #  return df

In [138]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)]
        
    return df

In [139]:
df = get_temperature_diff_between(original_df,1900,2013)
df.head()

,year,Country,continent,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff,Code,gdp
12786,1960,AFGHANISTAN,Asia,13.985417,0.441833,33.00,65.00,2.253063e+06,-0.167167,AFG,5.377778e+08
12788,1960,ALGERIA,Africa,23.504083,0.452083,28.00,3.00,5.944026e+07,0.170417,DZA,2.723593e+09
12797,1960,AUSTRALIA,Oceania,21.210500,0.222000,-27.00,133.00,2.054328e+09,-0.887667,AUS,1.860679e+10
12798,1960,AUSTRIA,Europe,6.527083,0.308917,47.33,13.33,1.930452e+09,-0.321667,AUT,6.592694e+09
12802,1960,BANGLADESH,Asia,25.484250,0.272333,24.00,90.00,4.051911e+06,0.495333,BGD,4.274894e+09


In [140]:
df["gdp"].unique()

array([5.37777811e+08, 2.72359338e+09, 1.86067869e+10, ...,
       1.71222025e+11, 2.80372395e+10, 1.90910200e+10])

In [141]:
df["gdp_test"] = df["gdp"]*2

In [142]:
df

,year,Country,continent,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff,Code,gdp,gdp_test
12786,1960,AFGHANISTAN,Asia,13.985417,0.441833,33.00,65.00,2.253063e+06,-0.167167,AFG,5.377778e+08,1.075556e+09
12788,1960,ALGERIA,Africa,23.504083,0.452083,28.00,3.00,5.944026e+07,0.170417,DZA,2.723593e+09,5.447187e+09
12797,1960,AUSTRALIA,Oceania,21.210500,0.222000,-27.00,133.00,2.054328e+09,-0.887667,AUS,1.860679e+10,3.721357e+10
12798,1960,AUSTRIA,Europe,6.527083,0.308917,47.33,13.33,1.930452e+09,-0.321667,AUT,6.592694e+09,1.318539e+10
12802,1960,BANGLADESH,Asia,25.484250,0.272333,24.00,90.00,4.051911e+06,0.495333,BGD,4.274894e+09,8.549788e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
24334,2013,URUGUAY,South America,16.754375,0.348250,-33.00,-56.00,3.261600e+08,-1.754625,URY,5.753123e+10,1.150625e+11
24335,2013,UZBEKISTAN,Asia,16.188250,0.350500,41.00,64.00,5.622211e+09,3.044083,UZB,7.318004e+10,1.463601e+11
24337,2013,VIETNAM,Asia,25.232125,0.383125,16.00,106.00,2.275257e+09,0.527792,VNM,1.712220e+11,3.424441e+11
24340,2013,ZAMBIA,Africa,21.196000,0.825125,-15.00,30.00,1.693902e+08,-0.501750,ZMB,2.803724e+10,5.607448e+10


In [143]:
fig = px.scatter(df, x="temp_diff", y="GHG", color="continent", template="simple_white", hover_name="Country",
                 size = "gdp_test",size_max=80,
                labels={
                     "temp_diff": "Increase in average temperature",
                     "GHG": "Greenhouse Gases Emissions"

                 },)
fig.show(renderer = "browser")

In [144]:
df.to_csv("Correlation.csv")

In [30]:
#def get_temperature_diff_between(df_by_year,start_date,end_date):
   # df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
             #           (df_by_year["year"] <= end_date)].groupby(["Country"]).sum()[["temp_diff"]]
    
    #return df